In [3]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

In [4]:
random_seed = 42
lr = 1e-3
batch_size = 32
n_epochs = 15
img_size = 224
n_classes = 10

In [33]:
def get_accuracy(model, data_loader, device):
    correct_pred = 0
    n = 0
    
    with torch.no_grad():
        model.eval()
        for X, y_true in data_loader:
            X, y_true = X.to(device), y_true.to(device)
            
            y_pred = model(X)
            _, predicted_labels = torch.max(y_pred, 1)
            
            n += y_true.size(0)
            correct_pred += (predicted_labels == y_true).sum()
    
    return correct_pred.float() / n

In [30]:
def train(train_loader, model, criterion, optimizer, device):
    model.train()
    running_loss = 0
    for X, y_true in train_loader:
        optimizer.zero_grad()
        
        X, y_true = X.to(device), y_true.to(device)
        y_hat = model(X)
        loss = criterion(y_hat, y_true)
        running_loss += loss.item() * X.size(0)
        loss.backward()
        optimizer.step()
        
    epoch_loss = running_loss / len(train_loader.dataset)
    return model, optimizer, epoch_loss

In [31]:
def validation(valid_loader, model, criterion, device):
    model.eval()
    running_loss = 0
    for X, y_true in valid_loader:
        X, y_true = X.to(device), y_true.to(device)
        y_hat = model(X)
        loss = criterion(y_hat, y_true)
        running_loss += loss.item() * X.size(0)
    
    epoch_loss = running_loss / len(valid_loader.dataset)
    
    return model, epoch_loss

In [35]:
def training_loop(model, criterion, optimizer, train_loader, valid_loader, epochs, device, print_every = 1):
    best_loss = 1e10
    train_losses = []
    valid_losses = []
    
    for epoch in range(epochs):
        model, optimizer, train_loss = train(train_loader, model, criterion, optimizer, device)
        train_losses.append(train_loss)
        
        with torch.no_grad():
            model, valid_loss = validation(valid_loader, model, criterion, device)
            valid_losses.append(valid_loss)
        
        if epoch % print_every == (print_every - 1):
            train_acc = get_accuracy(model, train_loader, device)
            valid_acc = get_accuracy(model, valid_loader, device)
            
            print(f'{datetime.now().time().replace(microsecond=0)} --- '
                  f'Epoch: {epoch}\t'
                  f'Train loss: {train_loss:.4f}\t'
                  f'Valid loss: {valid_loss:.4f}\t'
                  f'Train accuracy: {100 * train_acc:.2f}\t'
                  f'Valid accuracy: {100 * valid_acc:.2f}')
          
    return model, optimizer, (train_losses, valid_losses)

In [15]:
common_transforms = transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize(224),
                transforms.Normalize([meanR, meanG, meanB],
                                     [stdR, stdG, stdB])
])

In [13]:
train_dataset = datasets.CIFAR10(root = 'cifar10',
                                 train = True,
                                 transform = common_transforms,
                                 download = True)

test_dataset = datasets.CIFAR10(root = 'cifar10',
                                train = False,
                                transform = common_transforms,
                                download = True)

0it [00:00, ?it/s]

Extracting cifar10\cifar-10-python.tar.gz to cifar10
Files already downloaded and verified


In [14]:
import numpy as np
meanRGB = [np.mean(x.numpy(), axis = (1, 2)) for x, _ in train_dataset]
stdRGB = [np.std(x.numpy(), axis = (1, 2)) for x, _ in train_dataset]

meanR = np.mean([m[0] for m in meanRGB])
meanG = np.mean([m[1] for m in meanRGB])
meanB = np.mean([m[2] for m in meanRGB])

stdR = np.mean([m[0] for m in stdRGB])
stdG = np.mean([m[1] for m in stdRGB])
stdB = np.mean([m[2] for m in stdRGB])

In [16]:
common_transforms = transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize(224),
                transforms.Normalize([meanR, meanG, meanB],
                                     [stdR, stdG, stdB])
])

In [19]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 0)
indices = list(range(len(test_dataset)))
y_test0 = [y for _, y in test_dataset]

for test_index, val_index in sss.split(indices, y_test0):
    print('test :', len(test_index) , 'val :', len(val_index))    

test : 8000 val : 2000


In [20]:
from torch.utils.data import Subset

valid_dataset = Subset(test_dataset, val_index)
test_dataset = Subset(test_dataset, test_index)

In [21]:
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
valid_loader = DataLoader(valid_dataset, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [22]:
import torch
import torch.nn as nn

class GoogLeNet(nn.Module):
    def __init__(self, in_channels = 3, num_classes = 1000):
        super(GoogLeNet, self).__init__()

        self.conv1 = conv_block(in_channels = in_channels, out_channels = 64,
                                kernel_size = (7, 7), stride = (2, 2), padding = (3, 3))
        self.maxpool1 = nn.MaxPool2d(kernel_size= 3, stride = 2, padding = 1)

        self.conv2 = conv_block(64, 192, kernel_size = 3, padding = 1, stride = 1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception3a = Inception_block(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception_block(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride = 2, padding = 1)

        self.inception4a = Inception_block(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception_block(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception_block(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception_block(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception_block(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride = 2, padding = 1)

        self.inception5a = Inception_block(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception_block(832, 384, 192, 384, 48, 128, 128)
        
        self.avgpool = nn.AvgPool2d(7, 1)
        self.dropout = nn.Dropout(p = 0.4)
        self.fc1 = nn.Linear(1024, num_classes)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)

        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)

        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)

        x = self.inception5a(x)
        x = self.inception5b(x)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.dropout(x)
        x = self.fc1(x)

        return x

class Inception_block(nn.Module):
    def __init__(self, in_channels, out_1x1, red_3x3, out_3x3, red_5x5, out_5x5, out_1x1pool):
        super(Inception_block, self).__init__()
        
        self.branch1 = conv_block(in_channels, out_1x1, kernel_size = 1)
        self.branch2 = nn.Sequential(
            conv_block(in_channels, red_3x3, kernel_size = 1),
            conv_block(red_3x3, out_3x3, kernel_size = 3, padding = 1)
        )
        
        self.branch3 = nn.Sequential(
            conv_block(in_channels, red_5x5, kernel_size = 1),
            conv_block(red_5x5, out_5x5, kernel_size = 5,  padding = 2)
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride = 1, padding = 1),
            conv_block(in_channels, out_1x1pool, kernel_size = 1)
        )

    def forward(self, x):
        # N x filters x 28 x 28
        return torch.cat([self.branch1(x), self.branch2(x), self.branch3(x), self.branch4(x)], 1)



class conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(conv_block, self).__init__()
        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(in_channels = in_channels, out_channels = out_channels, **kwargs)
                            # kernel_size = (1, 1), (3, 3), (5,5 )
        self.batchnorm = nn.BatchNorm2d(out_channels)
    
    def forward(self, x):
        return self.relu(self.batchnorm(self.conv(x)))

In [36]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = GoogLeNet(num_classes= 10).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
criterion = nn.CrossEntropyLoss()

model, optimizer, _ = training_loop(model, criterion, optimizer, train_loader, valid_loader, 15, device)

18:20:38 --- Epoch: 0	Train loss: 1.4679	Valid loss: 1.1314	Train accuracy: 60.79	Valid accuracy: 60.90
18:23:11 --- Epoch: 1	Train loss: 0.9849	Valid loss: 0.8733	Train accuracy: 72.40	Valid accuracy: 68.80
18:25:41 --- Epoch: 2	Train loss: 0.7544	Valid loss: 0.7010	Train accuracy: 78.85	Valid accuracy: 75.85
18:28:11 --- Epoch: 3	Train loss: 0.6214	Valid loss: 0.6285	Train accuracy: 83.98	Valid accuracy: 77.75
18:30:40 --- Epoch: 4	Train loss: 0.5217	Valid loss: 0.5568	Train accuracy: 86.72	Valid accuracy: 80.35
18:33:10 --- Epoch: 5	Train loss: 0.4428	Valid loss: 0.5741	Train accuracy: 88.11	Valid accuracy: 80.95
18:35:41 --- Epoch: 6	Train loss: 0.3777	Valid loss: 0.5383	Train accuracy: 90.89	Valid accuracy: 82.75
18:38:07 --- Epoch: 7	Train loss: 0.3287	Valid loss: 0.6334	Train accuracy: 89.15	Valid accuracy: 80.15
18:40:34 --- Epoch: 8	Train loss: 0.2717	Valid loss: 0.5664	Train accuracy: 92.49	Valid accuracy: 81.70
18:43:01 --- Epoch: 9	Train loss: 0.2313	Valid loss: 0.4995	Trai